# 🚗 Predicting Used Car Prices

### 🔍 **Problem Statement**
Determine the factors that most strongly influence the sale price of a used car and provide price recommendations that maximize dealership profit while staying competitive in the local market.

---

### 💼 **Business Objective**
**Maximize gross profit and inventory turnover** by pricing each vehicle within a data-driven range that reflects what local buyers are willing to pay.

---

### 📊 **Analytics Objective**
Develop a **pricing-assist notebook** that delivers:  
1. **Best-guess sale price** (point estimate).  
2. **90 % prediction interval** (lower & upper bounds).  
3. **Feature-driven explanations** of what drives each estimate.

---

### ✅ **Success Criteria**
**Mean Absolute Error (MAE) ≤ \$1200** on a hold-out test set.

> **Why \$1200?**  
> - Typical gross profit per used car in the U.S. is **\$1200 – \$2300**.  
> - Keeping MAE at \$1200 caps pricing error so it never wipes out an average unit’s profit.  
> - This threshold corresponds to roughly **5 % MAPE** on an average retail price of \$25–26k, keeping errors within an acceptable profit band.

# 🤔 Understanding the Dataset

Our dataset contains **426k used-car records** with the following attributes:

- **id** – unique listing identifier  
- **region** – metro area where the car is listed  
- **price** – final transaction price in USD  
- **year** – model year of the vehicle  
- **manufacturer** – brand (e.g., Ford, Honda)  
- **model** – specific vehicle line (e.g., Civic, F-150)  
- **condition** – seller-reported condition category  
- **cylinders** – engine cylinder count (numeric or string)  
- **fuel** – fuel type (gas, diesel, hybrid, etc.)  
- **odometer** – mileage reading in miles  
- **title_status** – title state (clean, salvage, rebuilt, etc.)  
- **transmission** – automatic, manual, or other  
- **VIN** – Vehicle Identification Number  
- **drive** – drivetrain (FWD, RWD, 4WD/AWD)  
- **size** – vehicle size class (compact, full-size, etc.)  
- **type** – body style (SUV, sedan, truck, etc.)  
- **paint_color** – exterior color  
- **state** – U.S. state where listed  

### What we’ll do in this step
1. **Validate & assess data quality**  
   * Check for duplicate `id`/`VIN` entries, missing values, and outliers (e.g., negative prices, zero mileage).  
   * Standardize categorical levels and spot obvious typos.
2. **Explore initial distributions**  
   * Visualize price, year, and odometer histograms.  
   * Summarize top manufacturers, models, fuel types, and conditions.
3. **Spot early relationships**  
   * Quick correlations (price vs. mileage and age).  
   * Box plots of price by condition, transmission, and drive type.  

In [1]:
import pandas as pd
import numpy as np
import utils

# Load the dataset
df = pd.read_csv('../data/vehicles.csv')

# Print basic info about the dataset
print("="*50)
print("DATASET INFO")
print("="*50)
df.info()

DATASET INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  obje

In [2]:
print("="*50)
print("FIRST 5 ROWS")
print("="*50)
df.head()

FIRST 5 ROWS


,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
0,7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc


Let's analyze the `NaNs` situation

In [3]:
# Calculate percentage of null values for each feature
null_percentages = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)

print("="*60)
print("NULL VALUES PERCENTAGE BY FEATURE (DESCENDING ORDER)")
print("="*60)

for feature, null_pct in null_percentages.items():
    null_count = df[feature].isnull().sum()
    print(f"{feature:<15} | {null_count:>7,} nulls | {null_pct:>6.2f}%")

print(f"\nTotal dataset size: {len(df):,} records")


NULL VALUES PERCENTAGE BY FEATURE (DESCENDING ORDER)
size            | 306,361 nulls |  71.77%
cylinders       | 177,678 nulls |  41.62%
condition       | 174,104 nulls |  40.79%
VIN             | 161,042 nulls |  37.73%
drive           | 130,567 nulls |  30.59%
paint_color     | 130,203 nulls |  30.50%
type            |  92,858 nulls |  21.75%
manufacturer    |  17,646 nulls |   4.13%
title_status    |   8,242 nulls |   1.93%
model           |   5,277 nulls |   1.24%
odometer        |   4,400 nulls |   1.03%
fuel            |   3,013 nulls |   0.71%
transmission    |   2,556 nulls |   0.60%
year            |   1,205 nulls |   0.28%
id              |       0 nulls |   0.00%
region          |       0 nulls |   0.00%
price           |       0 nulls |   0.00%
state           |       0 nulls |   0.00%

Total dataset size: 426,880 records


### 7 features with significant NaNs

Based on the table above, there are 7 features with significant amounts of missing info:
- size: suggest dropping (values aren't very useful for pricing prediction). ❌ Drop feature
- cylinders: probably not highly important. ❌ Drop feature
- condition: critical feature to predict price. ✅ Keep
- VIN: Drop this feature, not  relevant for pricing model. ❌ Drop feature
- drive: suggest dropping (values not very useful for pricing). ❌ Drop feature
- paint_color: probably an important feature. ✅ Keep
- type: probably an important feature. ✅ Keep

Lets check the values of the 3 relevant features with NaNs: `condition`, `paint_color`, `type`

In [4]:
# Analyze value counts for 'condition' column
result = utils.analyze_column(df, 'condition')

VALUE COUNTS FOR 'CONDITION' COLUMN
Non-null records: 252,776 out of 426,880 total records
Null percentage: 40.79%

Top 6 values:
   1. good                 | 121,456 ( 48.0%)
   2. excellent            | 101,467 ( 40.1%)
   3. like new             | 21,178 (  8.4%)
   4. fair                 |  6,769 (  2.7%)
   5. new                  |  1,305 (  0.5%)
   6. salvage              |    601 (  0.2%)

Total unique values: 6




In [5]:
# Analyze value counts for 'paint_color' column
result = utils.analyze_column(df, 'paint_color')

VALUE COUNTS FOR 'PAINT_COLOR' COLUMN
Non-null records: 296,677 out of 426,880 total records
Null percentage: 30.50%

Top 10 values:
   1. white                | 79,285 ( 26.7%)
   2. black                | 62,861 ( 21.2%)
   3. silver               | 42,970 ( 14.5%)
   4. blue                 | 31,223 ( 10.5%)
   5. red                  | 30,473 ( 10.3%)
   6. grey                 | 24,416 (  8.2%)
   7. green                |  7,343 (  2.5%)
   8. custom               |  6,700 (  2.3%)
   9. brown                |  6,593 (  2.2%)
  10. yellow               |  2,142 (  0.7%)
  ... and 2 more unique values

Total unique values: 12




In [6]:
# Analyze value counts for 'type' column
result = utils.analyze_column(df, 'type')

VALUE COUNTS FOR 'TYPE' COLUMN
Non-null records: 334,022 out of 426,880 total records
Null percentage: 21.75%

Top 10 values:
   1. sedan                | 87,056 ( 26.1%)
   2. SUV                  | 77,284 ( 23.1%)
   3. pickup               | 43,510 ( 13.0%)
   4. truck                | 35,279 ( 10.6%)
   5. other                | 22,110 (  6.6%)
   6. coupe                | 19,204 (  5.7%)
   7. hatchback            | 16,598 (  5.0%)
   8. wagon                | 10,751 (  3.2%)
   9. van                  |  8,548 (  2.6%)
  10. convertible          |  7,731 (  2.3%)
  ... and 3 more unique values

Total unique values: 13




> **Conclusion:** the values of the 3 columns `condition`, `paint_color`, `type` look like great candidates for a pricing model! Because of this, we will move forward by *including these columns*, and filling up the NaNs with `missing` value

In [7]:
# Create cleaned dataframe with dropna for year/odometer
print("="*70)
print("CREATING CLEANED DATAFRAME")
print("="*70)

# Step 1: Drop unwanted columns
columns_to_drop = ['size', 'cylinders', 'VIN', 'drive']
df_cars = df.drop(columns=columns_to_drop)

print(f"Dropped columns: {columns_to_drop}")

# Step 2: Remove rows with missing year/odometer first
rows_before = len(df_cars)
year_nulls = df_cars['year'].isnull().sum()
odometer_nulls = df_cars['odometer'].isnull().sum()

# Remove rows with missing year or odometer
df_cars = df_cars.dropna(subset=['year', 'odometer'])

rows_after = len(df_cars)
rows_removed = rows_before - rows_after

print(f"year            | {year_nulls:>7,} rows with nulls removed")
print(f"odometer        | {odometer_nulls:>7,} rows with nulls removed")
print(f"\nTotal rows removed: {rows_removed:,}")

CREATING CLEANED DATAFRAME
Dropped columns: ['size', 'cylinders', 'VIN', 'drive']
year            |   1,205 rows with nulls removed
odometer        |   4,400 rows with nulls removed

Total rows removed: 5,536


In [8]:

# Step 3: Apply "missing" category to categorical features
print(f"{'='*50}")
print("APPLYING 'MISSING' CATEGORY TO CATEGORICAL NULL VALUES")
print(f"{'='*50}")

# Handle categorical columns (object type)
categorical_cols = df_cars.select_dtypes(include=['object']).columns.tolist()
# Remove id from categorical processing
categorical_cols = [col for col in categorical_cols if col not in ['id']]

for col in categorical_cols:
    null_count_before = df_cars[col].isnull().sum()
    df_cars[col] = df_cars[col].fillna('missing')
    print(f"{col:<15} | {null_count_before:>7,} nulls filled with 'missing'")

print(f"\n{'='*50}")
print("FINAL CLEANED DATAFRAME SUMMARY")
print(f"{'='*50}")
print(f"Final shape: {df_cars.shape}")

# Check for any remaining nulls
remaining_nulls = df_cars.isnull().sum().sum()
print(f"Total remaining null values: {remaining_nulls}")

APPLYING 'MISSING' CATEGORY TO CATEGORICAL NULL VALUES
region          |       0 nulls filled with 'missing'
manufacturer    |  16,267 nulls filled with 'missing'
model           |   5,195 nulls filled with 'missing'
condition       | 170,493 nulls filled with 'missing'
fuel            |   2,172 nulls filled with 'missing'
title_status    |   7,358 nulls filled with 'missing'
transmission    |   1,695 nulls filled with 'missing'
type            |  91,782 nulls filled with 'missing'
paint_color     | 128,090 nulls filled with 'missing'
state           |       0 nulls filled with 'missing'

FINAL CLEANED DATAFRAME SUMMARY
Final shape: (421344, 14)
Total remaining null values: 0


In [9]:
print(f"{'='*50}")
print("FIRST 5 ROWS OF CLEANED DATA")
print(f"{'='*50}")
df_cars.head()


FIRST 5 ROWS OF CLEANED DATA


,id,region,price,year,manufacturer,model,condition,fuel,odometer,title_status,transmission,type,paint_color,state
27,7316814884,auburn,33590,2014.0,gmc,sierra 1500 crew cab slt,good,gas,57923.0,clean,other,pickup,white,al
28,7316814758,auburn,22590,2010.0,chevrolet,silverado 1500,good,gas,71229.0,clean,other,pickup,blue,al
29,7316814989,auburn,39590,2020.0,chevrolet,silverado 1500 crew,good,gas,19160.0,clean,other,pickup,red,al
30,7316743432,auburn,30990,2017.0,toyota,tundra double cab sr,good,gas,41124.0,clean,other,pickup,red,al
31,7316356412,auburn,15000,2013.0,ford,f-150 xlt,excellent,gas,128000.0,clean,automatic,truck,black,al


## 🔡 Handling issues with high cardinality columns

2 columns exhibit high cardinality: `model` and `region`. I will analyze them to decide how to handle them.

In [10]:
# Analyze model column using utility function
model_analysis = utils.analyze_categorical_distribution(df_cars, 'model', top_n=40, related_column='manufacturer')


MODEL COLUMN DISTRIBUTION ANALYSIS
Total records in dataset: 421,344
Number of distinct models: 29,220
Average records per model: 14.4

Top 40 most common models (with manufacturer):
   1. f-150                     | ford            | 7,870 ( 1.9%)
   2. missing                   | N/A             | 5,195 ( 1.2%)
   3. silverado 1500            | chevrolet       | 5,043 ( 1.2%)
   4. 1500                      | ram             | 4,178 ( 1.0%)
   5. camry                     | toyota          | 3,070 ( 0.7%)
   6. silverado                 | chevrolet       | 2,991 ( 0.7%)
   7. accord                    | honda           | 2,925 ( 0.7%)
   8. wrangler                  | jeep            | 2,786 ( 0.7%)
   9. civic                     | honda           | 2,775 ( 0.7%)
  10. altima                    | nissan          | 2,719 ( 0.6%)
  11. escape                    | ford            | 2,709 ( 0.6%)
  12. 2500                      | ram             | 2,675 ( 0.6%)
  13. tacoma             

> The `model` column has ~30k distinct values. So, one-hot encoding them won't work. I've decided to **keep the top 50 models** (remaining ones will be set to `other_<manufacturer>`). These top 40 cover 22% of the dataset (meaning: for the remaining less popular ~78% models we will lose predictive power).

In [11]:
# Analyze region column using utility function
model_analysis = utils.analyze_categorical_distribution(df_cars, 'region', top_n=40, related_column='state')

REGION COLUMN DISTRIBUTION ANALYSIS
Total records in dataset: 421,344
Number of distinct regions: 404
Average records per region: 1042.9

Top 40 most common regions (with state):
   1. columbus                  | oh              | 3,608 ( 0.9%)
   2. jacksonville              | fl              | 3,522 ( 0.8%)
   3. spokane / coeur d'alene   | id              | 2,985 ( 0.7%)
   4. new hampshire             | nh              | 2,979 ( 0.7%)
   5. sarasota-bradenton        | fl              | 2,975 ( 0.7%)
   6. south jersey              | nj              | 2,973 ( 0.7%)
   7. eugene                    | or              | 2,973 ( 0.7%)
   8. grand rapids              | mi              | 2,972 ( 0.7%)
   9. pittsburgh                | pa              | 2,970 ( 0.7%)
  10. houston                   | tx              | 2,969 ( 0.7%)
  11. nashville                 | tn              | 2,969 ( 0.7%)
  12. philadelphia              | pa              | 2,967 ( 0.7%)
  13. rochester              

> The `region` column has 404 distinct values. So, one-hot encoding them won't work. I've decided to **keep the top 40 regions** (remaining ones will be set to `other_<state>`). These top 40 cover 28% of the dataset (meaning: for the remaining less popular ~72% regions we will lose predictive power).

### Simplifying the dataframe by reducing cardinality

In [12]:
# Start with a copy of the cleaned dataframe
df_cars_simplified = df_cars.copy()

# Process the 'model' column using the utility function
df_cars_simplified['model'] = utils.simplify_high_cardinality_column(
    df_cars_simplified, 
    target_column='model', 
    related_column='manufacturer', 
    top_n=40
)


SIMPLIFYING 'MODEL' COLUMN (TOP 40)
Top 40 models identified: 40 values
Top 10 models: ['f-150', 'missing', 'silverado 1500', '1500', 'camry', 'silverado', 'accord', 'wrangler', 'civic', 'altima'] ... (and 30 more)

Original unique models: 29,220
Simplified unique models: 82
Reduction: 29,138 values (99.7%)

New model distribution (top 15):
   1. other_ford                     | 47,650 (11.3%)
   2. other_chevrolet                | 33,365 ( 7.9%)
   3. other_toyota                   | 21,257 ( 5.0%)
   4. other_missing                  | 16,233 ( 3.9%)
   5. other_nissan                   | 15,978 ( 3.8%)
   6. other_gmc                      | 14,678 ( 3.5%)
   7. other_bmw                      | 14,494 ( 3.4%)
   8. other_jeep                     | 13,420 ( 3.2%)
   9. other_honda                    | 11,840 ( 2.8%)
  10. other_mercedes-benz            | 11,474 ( 2.7%)
  11. other_dodge                    |  9,959 ( 2.4%)
  12. other_ram                      |  9,490 ( 2.3%)
  13. oth

In [13]:
# Process the 'model' column using the utility function
df_cars_simplified['region'] = utils.simplify_high_cardinality_column(
    df_cars_simplified, 
    target_column='region', 
    related_column='state', 
    top_n=40
)

SIMPLIFYING 'REGION' COLUMN (TOP 40)
Top 40 regions identified: 40 values
Top 10 regions: ['columbus', 'jacksonville', "spokane / coeur d'alene", 'new hampshire', 'sarasota-bradenton', 'south jersey', 'eugene', 'grand rapids', 'pittsburgh', 'houston'] ... (and 30 more)

Original unique regions: 404
Simplified unique regions: 87
Reduction: 317 values (78.5%)

New region distribution (top 15):
   1. other_ca                       | 37,652 ( 8.9%)
   2. other_tx                       | 16,799 ( 4.0%)
   3. other_fl                       | 16,340 ( 3.9%)
   4. other_nc                       | 14,198 ( 3.4%)
   5. other_or                       | 13,759 ( 3.3%)
   6. other_oh                       | 11,722 ( 2.8%)
   7. other_ny                       | 11,135 ( 2.6%)
   8. other_mi                       | 10,932 ( 2.6%)
   9. other_co                       | 10,926 ( 2.6%)
  10. other_va                       | 10,511 ( 2.5%)
  11. other_wa                       | 10,400 ( 2.5%)
  12. other

In [15]:
df_cars_processed = df_cars_simplified.copy()

print("="*60)
print("FINAL PROCESSED DATAFRAME SUMMARY")
print("="*60)
print(f"Final shape: {df_cars_processed.shape}")
print(f"Features: {list(df_cars_processed.columns)}")
print(f"\nSample of simplified high-cardinality columns:")
print(f"Unique models: {df_cars_processed['model'].nunique()}")
print(f"Unique regions: {df_cars_processed['region'].nunique()}")

# Export to CSV
print(f"\n{'='*60}")
print("EXPORTING TO CSV")
print(f"{'='*60}")

output_path = '../data/vehicles_processed.csv'
df_cars_processed.to_csv(output_path, index=False)

print(f"✅ Successfully exported {len(df_cars_processed):,} records to '{output_path}'")
print(f"   File size: {df_cars_processed.memory_usage(deep=True).sum() / 1024**2:.1f} MB in memory")
print(f"   Ready for modeling! 🚗")

FINAL PROCESSED DATAFRAME SUMMARY
Final shape: (421344, 14)
Features: ['id', 'region', 'price', 'year', 'manufacturer', 'model', 'condition', 'fuel', 'odometer', 'title_status', 'transmission', 'type', 'paint_color', 'state']

Sample of simplified high-cardinality columns:
Unique models: 82
Unique regions: 87

EXPORTING TO CSV
✅ Successfully exported 421,344 records to '../data/vehicles_processed.csv'
   File size: 237.8 MB in memory
   Ready for modeling! 🚗
